In [2]:
import pandas as pd
from IPython.display import display, HTML
import re

In [3]:
import spacy
import en_core_web_lg
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer


In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hoang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hoang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df = pd.read_csv('../data/discharge.csv1.csv')

In [51]:
text_column = df['text']

In [6]:
# Accessing the first row's text data
first_row_text = df['text'].iloc[0]

# Print the full text data from the first row
print(first_row_text)


 
Name:  ___                     Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___
 
Chief Complaint:
Worsening ABD distension and pain 
 
Major Surgical or Invasive Procedure:
Paracentesis

 
History of Present Illness:
___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, 
bioplar, PTSD, presented from OSH ED with worsening abd 
distension over past week.  
Pt reports self-discontinuing lasix and spirnolactone ___ weeks 
ago, because she feels like "they don't do anything" and that 
she "doesn't want to put more chemicals in her." She does not 
follow Na-restricted diets. In the past week, she notes that she 
has been having worsening abd distension and discomfort. She 
denies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, 
dysuria. She had food poisoning a week ago from eating stale 
cake (n/v 20 min after fo

In [9]:
patterns = {
    "Chief Complaint": r"Chief Complaint:\s*(.+?)(?=Major Surgical|History of Present Illness|$)",
    "Major Surgical or Invasive Procedure": r"Major Surgical or Invasive Procedure:\s*(.+?)(?=History of Present Illness|$)",
    "Discharge Diagnosis": r"Discharge Diagnosis:\s*(.+?)(?=Discharge Condition|$)",
    "Discharge Medications": r"Discharge Medications:\s*(.+?)(?=Discharge Diagnosis|$)",
    "Discharge Condition": r"Discharge Condition:\s*(.+?)(?=Discharge Instructions|$)",
    "Discharge Instructions": r"Discharge Instructions:\s*(.+?)(?=Followup Instructions|$)",
    "Followup Instructions": r"Followup Instructions:\s*(.+)$"
}

# Extract sections using regex
data = {}
for key, pattern in patterns.items():
    match = re.search(pattern, first_row_text, re.DOTALL)
    if match:
        data[key] = match.group(1).strip()
    else:
        data[key] = None

# Convert the data dictionary into a pandas DataFrame
df_new = pd.DataFrame([data])

print(df_new)

                     Chief Complaint Major Surgical or Invasive Procedure  \
0  Worsening ABD distension and pain                         Paracentesis   

       Discharge Diagnosis                              Discharge Medications  \
0  Ascites from Portal HTN  1. Albuterol Inhaler 2 PUFF IH Q4H:PRN wheezin...   

                                 Discharge Condition  \
0  Mental Status: Clear and coherent.\nLevel of C...   

                              Discharge Instructions Followup Instructions  
0  Dear Ms. ___,\nIt was a pleasure taking care o...                   ___  


In [53]:
display(HTML(df.head(2).to_html()))

In [54]:
print(df)

             note_id  subject_id   hadm_id note_type  note_seq  \
0     10000032-DS-21    10000032  22595853        DS        21   
1     10000032-DS-22    10000032  22841357        DS        22   
2     10000032-DS-23    10000032  29079034        DS        23   
3     10000032-DS-24    10000032  25742920        DS        24   
4     10000084-DS-17    10000084  23052089        DS        17   
...              ...         ...       ...       ...       ...   
4995  10157674-DS-23    10157674  20032792        DS        23   
4996  10157674-DS-24    10157674  23431884        DS        24   
4997  10157674-DS-25    10157674  26870854        DS        25   
4998  10157674-DS-26    10157674  23743097        DS        26   
4999  10157674-DS-27    10157674  20023715        DS        27   

                charttime            storetime  \
0     2180-05-07 00:00:00  2180-05-09 15:26:00   
1     2180-06-27 00:00:00  2180-07-01 10:15:00   
2     2180-07-25 00:00:00  2180-07-25 21:42:00   
3     2

In [61]:
# Define columns to drop excluding 'note_id'
columns_to_drop = [col for col in df.columns if (col != 'text' and col != 'note_id')]
print("Columns to drop: ", columns_to_drop)


Columns to drop:  ['subject_id', 'hadm_id', 'note_type', 'note_seq', 'charttime', 'storetime']


In [62]:
df.drop(columns=columns_to_drop, inplace=True)
notes_data = df.set_index('note_id')

In [1]:
display(HTML(df.head(3).to_html()))

NameError: name 'HTML' is not defined

In [78]:
def clean_text(x):
    x = " ".join(x.split())
    x= " ".join((" ".join(x.split("[**"))).split("**]"))
    x = re.sub(r"\([^()]*\)", "", x)
    key_value_strip =(x.split(":"))
    ##remove all sub strings which have a length lesser than 50 characters
    string = " ".join([sub_unit for sub_unit in key_value_strip if len(sub_unit)>50])
    x = re.sub(r"(\d+)+(\.|\))", "", string)## remove all serialization eg 1. 1)
    x = re.sub(r"(\*|\?|=)+", "", x) ##removing all *, ? and =
    x = re.sub(r"\b(\w+)( \1\b)+", r"\1", x) ## removing consecutive dupicate words
    x = x.replace("FOLLOW UP", "FOLLOWUP")
    x = x.replace("FOLLOW-UP", "FOLLOWUP")
    x = re.sub(r"(\b)(f|F)(irst)(\b)?[\d\-\d]*(\s)*(\b)?(n|N)(ame)[\d\-\d]*(\s)*[\d\-\d]*(\b)","",x)##remove firstname
    x = re.sub(r"(\b)(l|L)(ast)(\b)?[\d\-\d]*(\s)*(\b)?(n|N)(ame)[\d\-\d]*(\s)*[\d\-\d]*(\b)", "", x)
    x = re.sub(r"(\b)(d|D)\.?(r|R)\.?(\b)", "", x) #remove Dr abreviation
    x = re.sub(r"([^A-Za-z0-9\s](\s)){2,}", "", x)##remove consecutive punctuations
    x = re.sub(r'___', '', x) # Remove occurrences of "___" using regex
    x = re.sub(r'. __.', '', x) # Remove occurrences of ". __." using regex
    x = x.replace('"', '')
    

    return(x.replace("  ", " "))


notes_data["text"] = notes_data["text"].apply(lambda x: clean_text(x))



In [79]:
display(HTML(notes_data.head(1).to_html()))

In [74]:
first_row_text = notes_data['text'].iloc[0]

# Print the full text data from the first row
print(first_row_text)

No Known Allergies / Adverse Drug Reactions Attending Worsening ABD distension and pain Major Surgical or Invasive Procedure HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. Pt reports self-discontinuing lasix and spirnolactone weeks ago, because she feels like "they don't do anything" and that she "doesn't want to put more chemicals in her." She does not follow Na-restricted diets. In the past week, she notes that she has been having worsening abd distension and discomfort. She denies edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, dysuria. She had food poisoning a week ago from eating stale cake , which resolved the same day. She denies other recent illness or sick contacts. She notes that she has been noticing gum bleeding while brushing her teeth in recent weeks. she denies easy bruising, melena, BRBPR, hemetesis, hemoptysis, or hematuria. Because of her abd pain, she went to OSH ED and w

In [80]:
topics =notes_data["text"].apply(lambda text:list(filter(None,[x.strip() for x in re.findall(r"\b[A-Z\s]+\b",text)])))
topics = topics.tolist() 
#combine all items from topics and find unique elements 
topics = sorted(set([sub_topic.strip() for topic in topics for sub_topic in topic]))
print("Number of unique topics identified: ",len(topics))

##We remove all words from the topics list which had a high potential to be an acronym
topics = [topic for topic in topics if (len(topic.split())>2) or (len(topic.split())<2 and len(topic)>4)]
print("Unique Eligible topics: ", len(topics))


Number of unique topics identified:  16945
Unique Eligible topics:  8360


In [81]:
# Convert the list of filtered topics to a DataFrame
topics_df = pd.DataFrame(topics, columns=['topics'])

# Define the file path where you want to save the CSV file
csv_file_path = "potential_topics.csv"

# Save the DataFrame to a CSV file
topics_df.to_csv(csv_file_path, index=False)

print("Filtered topics saved to:", csv_file_path)

Filtered topics saved to: potential_topics.csv


In [86]:
topics = pd.read_csv('../notebook.py/potential_topics.csv')

In [88]:
print(topics.columns)


Index(['topics'], dtype='object')


In [89]:
nlp = en_core_web_lg.load()
stemmer = SnowballStemmer(language='english')

In [101]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [102]:
!pip -q install transformers torch accelerate

In [107]:

# Use a pipeline as a high-level helper
from transformers import pipeline
summarizer = pipeline('summarization')
input_text = "summary this text: " + first_row_text
print(input_text)
summarizer(input_text, max_length = 400, min_length = 150, do_sample = False)


In [108]:
summarizer = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\hoang\Desktop\LLMs project\venv\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hoang\.cache\huggingface\hub\models--sshleifer--distilbart-cnn-12-6. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate develop

In [105]:
input_text = "summary this text: " + first_row_text
print(input_text)

summary this text: No Known Allergies / Adverse Drug Reactions Attending Worsening ABD distension and pain Major Surgical or Invasive Procedure HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. Pt reports self-discontinuing lasix and spirnolactone weeks ago, because she feels like "they don't do anything" and that she "doesn't want to put more chemicals in her." She does not follow Na-restricted diets. In the past week, she notes that she has been having worsening abd distension and discomfort. She denies edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, dysuria. She had food poisoning a week ago from eating stale cake , which resolved the same day. She denies other recent illness or sick contacts. She notes that she has been noticing gum bleeding while brushing her teeth in recent weeks. she denies easy bruising, melena, BRBPR, hemetesis, hemoptysis, or hematuria. Because of her abd pain, she w

In [110]:
summarizer(input_text, max_length = 400, min_length = 150, do_sample = False)

IndexError: index out of range in self